In [9]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

## Multivariate LR

In [4]:
class Mul_LR(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,1) # input, output
    
    def forward(self, x):
        return self.linear(x)

In [5]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
model = Mul_LR()

# optimizer 
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs):
    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/20 Cost: 31667.599609
Epoch    1/20 Cost: 9926.266602
Epoch    2/20 Cost: 3111.513916
Epoch    3/20 Cost: 975.451355
Epoch    4/20 Cost: 305.908539
Epoch    5/20 Cost: 96.042496
Epoch    6/20 Cost: 30.260748
Epoch    7/20 Cost: 9.641701
Epoch    8/20 Cost: 3.178671
Epoch    9/20 Cost: 1.152871
Epoch   10/20 Cost: 0.517863
Epoch   11/20 Cost: 0.318801
Epoch   12/20 Cost: 0.256388
Epoch   13/20 Cost: 0.236821
Epoch   14/20 Cost: 0.230660
Epoch   15/20 Cost: 0.228719
Epoch   16/20 Cost: 0.228095
Epoch   17/20 Cost: 0.227880
Epoch   18/20 Cost: 0.227799
Epoch   19/20 Cost: 0.227762


## Minibatch (Dataset, DataLoader)

In [6]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]
    
    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])

        return x, y

dataset = CustomDataset()        

dataloader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=True, 
    )

In [7]:
# 모델 초기화
model = Mul_LR()

# optimizer 
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples

        prediction = model(x_train)
        cost = F.mse_loss(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        # 로그 출력
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/20 Cost: 40394.078125
Epoch    0/20 Cost: 10625.849609
Epoch    0/20 Cost: 5451.569824
Epoch    1/20 Cost: 937.502686
Epoch    1/20 Cost: 265.944763
Epoch    1/20 Cost: 244.422516
Epoch    2/20 Cost: 3.523164
Epoch    2/20 Cost: 55.838959
Epoch    2/20 Cost: 2.371156
Epoch    3/20 Cost: 21.615038
Epoch    3/20 Cost: 1.536141
Epoch    3/20 Cost: 23.620941
Epoch    4/20 Cost: 12.848389
Epoch    4/20 Cost: 17.578531
Epoch    4/20 Cost: 18.382185
Epoch    5/20 Cost: 11.401794
Epoch    5/20 Cost: 20.960949
Epoch    5/20 Cost: 28.939528
Epoch    6/20 Cost: 14.807865
Epoch    6/20 Cost: 2.599264
Epoch    6/20 Cost: 33.501408
Epoch    7/20 Cost: 16.869854
Epoch    7/20 Cost: 24.528923
Epoch    7/20 Cost: 10.461182
Epoch    8/20 Cost: 8.686996
Epoch    8/20 Cost: 30.927486
Epoch    8/20 Cost: 7.897373
Epoch    9/20 Cost: 7.612912
Epoch    9/20 Cost: 18.703394
Epoch    9/20 Cost: 16.064274
Epoch   10/20 Cost: 12.797729
Epoch   10/20 Cost: 8.214132
Epoch   10/20 Cost: 23.461618
Epoch  

## Logistic Regression

In [11]:
xy = np.loadtxt('./Class Material/data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [13]:
class BinaryClassifer(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8,1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [15]:
model = BinaryClassifer()

optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs+1):
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(epoch, nb_epochs, cost.item(), accuracy * 100,))

Epoch    0/100 Cost: 0.691402 Accuracy 55.47%
Epoch   10/100 Cost: 0.576110 Accuracy 66.93%
Epoch   20/100 Cost: 0.541669 Accuracy 72.33%
Epoch   30/100 Cost: 0.521164 Accuracy 76.28%
Epoch   40/100 Cost: 0.508066 Accuracy 76.55%
Epoch   50/100 Cost: 0.499238 Accuracy 77.08%
Epoch   60/100 Cost: 0.493040 Accuracy 77.08%
Epoch   70/100 Cost: 0.488553 Accuracy 77.08%
Epoch   80/100 Cost: 0.485224 Accuracy 76.42%
Epoch   90/100 Cost: 0.482704 Accuracy 76.42%
Epoch  100/100 Cost: 0.480765 Accuracy 76.81%


## Softmax Classification

In [16]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [17]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4,3)

    def forward(self, x):
        return self.linear(x)

model = SoftmaxClassifierModel()

optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))


Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609258
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358734
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269522
Epoch 1000/1000 Cost: 0.241922
